# Hydroscape Calculation Notebook (English Version)

This notebook demonstrates how to calculate **Hydroscape** metrics for each plot/species using measurements of leaf water potential at **predawn** (`Ψpd`) and **midday** (`Ψmd`).

The method follows the approach described in **Li et al. (2019)**, *More than iso/anisohydry: Hydroscapes integrate plant water use and drought tolerance traits in 10 eucalypt species from contrasting climates*.

For each plot, we:

1. Clean and standardize the data: convert water potentials to negative values (MPa), remove missing values and filter cases where `Ψmd` > `Ψpd`.
2. Fit a linear regression of `Ψmd` versus `Ψpd`.
3. Identify the subset of data points that maximizes the coefficient of determination (R2) while keeping the slope < 1.
4. Compute the intercept (a), slope (m), intersection with the 1:1 line (b), and Hydroscape area (`HS = (a * b) / 2`).


In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from pathlib import Path

# Define file paths
FILE_PATH = Path(r"C:\Users\varga\Box\Data\Water Potencials\WP.xlsx")
OUTPUT_DIR = Path(r"C:\Users\varga\Box\Data\Hydroscapes")

# Load the Excel file
df = pd.read_excel(FILE_PATH)

# Display the first few rows
print(df.head())


## Data cleaning and preprocessing

We convert `Ψpd` (`WP_PreDown`) and `Ψmd` (`WP_MidDay`) to negative values (if necessary), drop rows with missing values, and retain only observations where `Ψmd` ≤ `Ψpd`.


In [ ]:
# Ensure numeric types for water potentials
for col in ['WP_PreDown', 'WP_MidDay']:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Create negative-valued columns for water potentials
df['pd'] = -df['WP_PreDown'].abs()
df['md'] = -df['WP_MidDay'].abs()

# Drop rows with missing pd or md values
df_clean = df.dropna(subset=['pd', 'md']).copy()

# Filter data: md should be less than or equal to pd
condition = df_clean['md'] <= df_clean['pd']
df_clean = df_clean[condition]

print('Rows after cleaning:', len(df_clean))
df_clean[['Plot', 'pd', 'md']].head()


## Functions for regression and Hydroscape computation

We define helper functions:

- `best_subset_md_vs_pd(x, y, min_points=6)`: finds the subset of points with slope < 1 that maximizes R2.
- `hydroscape_from_fit(model)`: given a fitted regression model, computes the slope, intercept, intersection with the 1:1 line, and Hydroscape area.


In [ ]:
def best_subset_md_vs_pd(x, y, min_points=6):
    '''Return the best subset (in terms of highest R2) for md vs pd regression.
    Only subsets with at least min_points and slope < 1 are considered.'''
    # Sort by pd to ensure monotonic inclusion of data points
    idx = np.argsort(x.values)
    xv = x.values[idx]
    yv = y.values[idx]
    best = None
    for k in range(min_points, len(xv) + 1):
        X = sm.add_constant(xv[:k])
        model = sm.OLS(yv[:k], X).fit()
        slope = float(model.params[1])
        r2 = float(model.rsquared)
        # Accept subsets with slope < 1
        if slope < 1:
            if best is None or r2 > best['r2']:
                best = {'k': k, 'model': model, 'r2': r2}
    if best is None:
        # Fallback to using all data if no subset meets the criteria
        X = sm.add_constant(xv)
        model = sm.OLS(yv, X).fit()
        best = {'k': len(xv), 'model': model, 'r2': float(model.rsquared)}
    return best['model'], best['k'], best['r2']

def hydroscape_from_fit(model):
    '''Compute Hydroscape metrics from a fitted OLS model.'''
    m = float(model.params[1])
    a = float(model.params[0])
    # Intersection with 1:1 line (md = pd): a + m * x = x
    if abs(m - 1) < 1e-9:
        b = np.nan
        HS = np.nan
    else:
        b = (-a) / (m - 1)
        HS = (a * b) / 2.0
    return m, a, b, HS


## Compute Hydroscape metrics for each plot

We group the cleaned data by `Plot` and compute the Hydroscape metrics using the functions defined above. Results are stored in a dataframe and displayed below.


In [ ]:
results = []
for plot_name, grp in df_clean.groupby('Plot'):
    # Use only groups with at least 6 observations
    if len(grp) < 6:
        continue
    model, k, r2 = best_subset_md_vs_pd(grp['pd'], grp['md'])
    slope, intercept, b_intersect, hydroscape_area = hydroscape_from_fit(model)
    results.append({
        'Plot': plot_name,
        'n_used': k,
        'Slope': slope,
        'Intercept': intercept,
        'b_intersect': b_intersect,
        'Hydroscape': hydroscape_area,
        'R2': r2
    })

summary = pd.DataFrame(results)

# Display summary
summary


## Save results

We save the summary of Hydroscape metrics to the output directory specified by `OUTPUT_DIR`. If the directory does not exist, you may need to create it before running this code in your own environment.


In [ ]:
# Define output file path
output_path = OUTPUT_DIR / 'hydroscape_output.csv'

# Save summary to CSV
summary.to_csv(output_path, index=False)

print(f'Saved hydroscape metrics to {output_path}')
